In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import quad
from scipy.stats import norm
from scipy.optimize import newton
import scipy.optimize as opt
import math
from mpl_toolkits.mplot3d import Axes3D

HESTON MODEL

Parameters

lambda  : mean-reversion speed \
vbar    : long-term average volatility \
eta     : volatility of vol process \
rho     : correlation between stock and vol \
v0      : initial volatility \
r       : risk-free interest rate \
tau     : time to maturity \
S0      : initial share price \
K       : strike price 

In [ ]:
def PIntegrand(u, lambda_, vbar, eta, rho, v0, r, tau, S0, K, j):
        F = S0 * np.exp(r * tau)
        x = np.log(F / K)
        a = lambda_ * vbar

        if j == 1:
            b = lambda_ - rho * eta
            alpha = -u**2 / 2 - u / 2 * 1j + 1j * u
            beta = lambda_ - rho * eta - rho * eta * 1j * u
        else:
            b = lambda_
            alpha = -u**2 / 2 - u / 2 * 1j
            beta = lambda_ - rho * eta * 1j * u

        gamma = eta**2 / 2
        d = np.sqrt(beta**2 - 4 * alpha * gamma)
        rplus = (beta + d) / (2 * gamma)
        rminus = (beta - d) / (2 * gamma)
        g = rminus / rplus

        D = rminus * (1 - np.exp(-d * tau)) / (1 - g * np.exp(-d * tau))
        C = lambda_ * (rminus * tau - (2 / (eta**2)) * np.log((1 - g * np.exp(-d * tau)) / (1 - g)))

        top = np.exp(C * vbar + D * v0 + 1j * u * x)
        bottom = 1j * u
        real_part = (top / bottom).real
        return real_part

In [ ]:
def P(lambda_, vbar, eta, rho, v0, r, tau, S0, K, j):
        value, _ = quad(PIntegrand, 0, np.inf, args=(lambda_, vbar, eta, rho, v0, r, tau, S0, K, j))
        prob = 0.5 + (1/np.pi) * value
        return prob

In [ ]:
def HestonCallClosedForm(lambda_, vbar, eta, rho, v0, r, tau, S0, K):

    A = S0 * P(lambda_, vbar, eta, rho, v0, r, tau, S0, K, 1)
    B = K * np.exp(-r * tau) * P(lambda_, vbar, eta, rho, v0, r, tau, S0, K, 0)
    
    return A - B